# Flow ML Course

## TP 4 : Vertex AI  SDK Python

L'objectif de ce notebook est d'apprendre à utiliser le Model Registry de Vertex AI.

Un Model Registry permeter de créer et de versionner un modèles. Cela permet également de le déployer facilement et de mettre à disposition un endpoint pour requêter le modèle.

En savoir plus [Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction#:~:text=The%20Vertex%20AI%20Model%20Registry%20is%20a%20central,can%20better%20organize%2C%20track%2C%20and%20train%20new%20versions.).


Nous procéderons comme suit :
- Créer et sauver 2 version d'un modèle sur le Model Registry
- Mettre à jour la version du modèle
- Supprimer une version
- Ré-entrainer une autre version du modèle

Nous utiliserons ce type de modèle pour ce TP : [ResNet V2 pretained model](https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5). 

### Model

This tutorial uses a pre-trained image classification model from TensorFlow Hub, which is trained on ImageNet dataset.



In [ ]:
# Install the packages
USER=''
! pip3 install {USER} --upgrade --quiet google-cloud-aiplatform \
                                        tensorflow==2.15.1 \
                                        tensorflow-hub \
                                        tf_keras

In [ ]:
PROJECT_ID = "projet-ia-448520" # "[your-project-id]"  # @param {type:"string"}


# Set the project id
! gcloud config set project {PROJECT_ID}

LOCATION = "us-central1"  # @param {type:"string"}

BUCKET_URI = "gs://cours1bucket"

In [ ]:
import os

import google.cloud.aiplatform as aip
import tensorflow as tf
import tensorflow_hub as hub
import tf_keras

aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

##  Model Registry

Vertex AI Model Registry permet de regrouper plusieurs versions d'un modèle. Cela a pour but de : 

- Suivre les évolutions du model lineage
- Définir la version par défaut mais aussi de déterminer d'autres types de version via les alias (development, staging, production) 
- Déployer un modèle 





## Extraction des modèles

In [ ]:
tfhub_model_v1 = tf_keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_101/classification/5")]
)

tfhub_model_v1.build([None, 224, 224, 3])

tfhub_model_v1.summary()

In [ ]:
tfhub_model_v2 = tf_keras.Sequential(
    [hub.KerasLayer("https://www.kaggle.com/models/google/resnet-v2/TensorFlow2/50-classification/2")]
)

tfhub_model_v2.build([None, 128, 128, 3])

tfhub_model_v2.summary()

### A vous de jouer
Quelles sont les différences entre les modèles v1 & v2 ?

Votre Réponse : 

### Sauvegarder les modèles artifacts

Nous utiliserons google cloud Storage pour stocker nos modèles.

Utiliser un sous bucket à votre nom pour avoir votre propre répertoire.

In [ ]:
MODEL_V1_DIR = BUCKET_URI + ... + "/model/v1"
tfhub_model_v1.save(MODEL_V1_DIR)

MODEL_V2_DIR = BUCKET_URI + ... + "/model/v2"
tfhub_model_v2.save(MODEL_V2_DIR)

## Image de déploiement

Image de python permettant de faire tourner le modèle.

Ici une image spécifique à tensorflow pour usage CPU

In [ ]:
TF = "2.13".replace(".", "-")

DEPLOY_VERSION = "tf2-cpu.{}".format(TF) # Possibility to use GPU 


DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_VERSION)

### Upload version 1 

TensorFlow Hub model --> Vertex AI model resource

Uploadons la première version du modèle (`MODEL_DIR_V1`) dans le Model Registry avec les paramètres suivants :

- `is_default_version`: (bool) version par défault du modèle (pour le déploiement).
- `version_ailiases`: Alias name (exemple : `production`).
- `version_description`: Description Utilsiateur.

Une fois créer, aller dans le Model Registry (interface web) pour vérifier la `version_id`.

In [ ]:
your_name = "test"

model_v1 = aip.Model.upload(
    display_name=f"model_course_1_4_{your_name}",
    artifact_uri=MODEL_V1_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    is_default_version=True,
    version_aliases=["v1"],
    version_description=..., # Add a description
)

print(model_v1)

### Upload version 2 of the TensorFlow Hub model to a Vertex AI model resource

### Upload version 2

TensorFlow Hub model --> Vertex AI model resource

Uploadons la second version du modèle (`MODEL_DIR_V2`) dans le Model Registry avec les paramètres suivants :

- `parent_model`: Le modèle par défaut a associé aux autres versions (ici : `parent_model=model_v1.resource_name`)
- `is_default_version`: (bool) version par défault du modèle (pour le déploiement).
- `version_ailiases`: Alias name (exemple : `development`).
- `version_description`: Description Utilsiateur.

Une fois créer, aller dans le Model Registry (interface web) pour vérifier la `version_id`.


In [ ]:
model_v2 = ... # Upload the model with parameter : parent_model=model_v1.resource_name

print(model_v2)

### Récupérer les version d'un modèle Get all versions of the parent model

Lister les version du modèle parents via la méthode : `version_registry.list_versions()`.

In [ ]:
versions = model_v1.versioning_registry.list_versions()
for version in versions:
    print(version)

Répéter la même action sur le modèle non-parent :

In [ ]:
versions = model_v2.versioning_registry.list_versions()
for version in versions:
    print(version)

### Lister les ressources du modèle

Utiliser la méthode `list()` pour récupérer la resource du modèle.

In [ ]:
models = aip.Model.list(filter=f"display_name=model_course_1_4_{your_name}")
print("Number of models:", len(models))
print("Version ID:", models[0].version_id)

model = models[0]

Quelle version avez vous récupéré ?

Version par défault (#2)

### Changer la version par défault

Pour changer la version par défault d'un modèle, on peut utiliser la méthode : `versioning_registry.add_version_aliases()` 

- `version`: version_id 
- `new_aliases`: L'alias à assigner à la version désigner

Pour changer le modèle par défault, paramétrer la méthode avec ces paramètres `{new_aliases=["default"], version="1"}`. Valider ensuite que la version par défault du modèle a bien changé.


In [ ]:
model_v2.versioning_registry.add_version_aliases(new_aliases=["default"], version="1")

models = aip.Model.list(filter=f"display_name=model_course_1_4_{your_name}")
print("Number of models:", len(models))
print("Version ID:", models[0].version_id)

model = models[0]

## Créer un Endpoint

Vous explorerez plus en détail la création et l'utilisation d'endpoint dans la prochaine séance.

Regardons comment créer un endpoint : `Endpoint.create()` avec les paramètres suivants:


- `display_name`: nom de la resource 
- `project`: project ID.
- `location`: region projet.
- `labels`: (optionnel) métadonnées

La méthode renvoie un endpoint.

Pour approfondir [Vertex AI endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api).

In [ ]:
endpoint = aip.Endpoint.create(
    display_name=f"endpoint_course_1_4_{your_name}",
    project=PROJECT_ID,
    location=LOCATION,
    labels={"your_key": "your_value"},
)

print(endpoint)

## Le Compute

In [ ]:
DEPLOY_COMPUTE = "n1-standard-4"
print("Train machine type", DEPLOY_COMPUTE)

## Déploiement d'un modèle sur un endpoint 

L'objectif du endpoint est de mettre à disposition un modèle pour réaliser les prévisions. Le déploiement utilisera les ressource du Vertex AI Model et du container de déploiement spécifié. 

You can deploy one of more Vertex AI model resource instances to the same endpoint. Each Vertex AI model resource that is deployed has its own deployment container for the serving binary. 

Voici les paramètres de déploiements : :

- `model`: Le modèle vertex AI
- `deployed_model_displayed_name`: Nom du modèle à déployé
- `machine_type`: Type de machine pour la VM de déploiement.

Le déploiement peut prendre quelques minutes.

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name=f"endpoint_course_1_4_{your_name}",
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint)

#### Récupérer les informations d'un déploiement 

Vous pouvez récupérer les paramètres des déploiements à partir du endpoint via la méthode suivante : 

In [ ]:
endpoint.gca_resource.deployed_models[0]

### Supprimer le déploiement

La méthode undeploy du endpoint permet de supprimer un déploiement à partir du `deployed_model_id` :

In [ ]:
deployed_model_id = endpoint.gca_resource.deployed_models[0].id
print(deployed_model_id)

endpoint.undeploy(deployed_model_id)

### Supprimer une version d'un modèle

Utiliser la méthode `versioning_registry.delete_version()` pour supprimer une version à partir de son id ou de son alias (paramètre : `version`)


In [ ]:
# Deletes a specific model version, as long as it isn't the default version.
model.versioning_registry.delete_version(...) # Choose the version


# Verify that this version is deleted
versions = model.versioning_registry.list_versions()
for version in versions:
    print(version)

Que se passe-t-il si vous essayer de supprimer le modèle par défaut ?

C'est impossible !

### Supprimer un modèle

Lors de la suppression d'un modèle, toutes les versions sont supprimés

In [ ]:
model.delete()